In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
from livecellx import segment
from livecellx import core
from livecellx.core import datasets
from livecellx.core.datasets import LiveCellImageDataset
from skimage import measure
from livecellx.core import SingleCellTrajectory, SingleCellStatic
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from livecellx.segment.detectron_utils import gen_cfg

from livecellx.segment.detectron_utils import (
    segment_detectron_wrapper,
    segment_images_by_detectron,
    convert_detectron_instance_pred_masks_to_binary_masks,
    convert_detectron_instances_to_label_masks,
)
from livecellx.segment.detectron_utils import (
    convert_detectron_instance_pred_masks_to_binary_masks,
    convert_detectron_instances_to_label_masks,
    segment_images_by_detectron,
    segment_single_img_by_detectron_wrapper,
)


dataset_dir_path = Path(
    "../datasets/test_data_STAV-A549/DIC_data"
)

mask_dataset_path = Path("../datasets/test_data_STAV-A549/mask_data")
mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
dic_dataset = LiveCellImageDataset(dataset_dir_path, ext="tif")

In [ ]:
dataset_dir_path = Path(
    "../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/XY16/"
)

mask_dataset_path = Path("../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/out/XY16/seg")

mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
time2url = {i: path for i, path in enumerate(time2url)}
dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

Convert label masks to single objects

In [ ]:
from livecellx.core.io_sc import process_scs_from_label_mask, prep_scs_from_mask_dataset

single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

In [ ]:
# for testing
# single_cells = single_cells[:10]

In [ ]:
len(single_cells)

In [ ]:
single_cells_by_time = {}
for cell in single_cells:
    if cell.timeframe not in single_cells_by_time:
        single_cells_by_time[cell.timeframe] = []
    single_cells_by_time[cell.timeframe].append(cell)

Visualize one single cell

In [ ]:
sc = single_cells[0]

fig, axes = plt.subplots(1, 4, figsize=(10, 5))
sc.show(ax=axes[0])
sc.show_mask(ax=axes[1])
sc.show_contour_img(ax=axes[2])
sc.show_contour_mask(ax=axes[3])

Calculate overlap between two single cells

In [ ]:
sc.show_mask(padding=200)

In [ ]:
sc.show_contour_mask(padding=200)

In [ ]:
sc1 = single_cells_by_time[0][0]
sc2 = single_cells_by_time[0][1]

def compute_overlap_bf(sc1, sc2):
    # calculate overlap
    img_shape = sc1.get_img().shape
    # TODO: add a helper function in single_cell to returna mask with only the current cell in it.
    mask1 = sc1.get_contour_mask(padding=np.max(img_shape)).astype(bool)
    mask2 = sc2.get_contour_mask(padding=np.max(img_shape)).astype(bool)

    overlap_area = np.logical_and(mask1, mask2).sum()
    iou = overlap_area / (mask1 | mask2).sum()
    return overlap_area, iou

def bbox_overlap(bbox1, bbox2):
    # calculate overlap
    x1_min, y1_min, x1_max, y1_max = bbox1
    x2_min, y2_min, x2_max, y2_max = bbox2
    x_overlap = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
    y_overlap = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
    overlap_area = x_overlap * y_overlap
    bbox1_area = (x1_max - x1_min) * (y1_max - y1_min)
    bbox2_area = (x2_max - x2_min) * (y2_max - y2_min)
    iou = overlap_area / (bbox1_area + bbox2_area - overlap_area)
    return overlap_area, iou

def compute_overlap(sc1: SingleCellStatic, sc2: SingleCellStatic):
    bbox1, bbox2 = sc1.get_bbox(), sc2.get_bbox()
    bbox_overlap_area, bbox_iou = bbox_overlap(bbox1, bbox2)
    if bbox_iou <= 0:
        return 0, 0

    merged_bbox = (min(bbox1[0], bbox2[0]), min(bbox1[1], bbox2[1]), max(bbox1[2], bbox2[2]), max(bbox1[3], bbox2[3]))
    # calculate overlap
    # TODO: add a helper function in single_cell to returna mask with only the current cell in it.
    mask1 = sc1.get_contour_mask(crop=False)[merged_bbox[0]:merged_bbox[2], merged_bbox[1]:merged_bbox[3]]
    mask2 = sc2.get_contour_mask(crop=False)[merged_bbox[0]:merged_bbox[2], merged_bbox[1]:merged_bbox[3]]
    overlap_area = np.logical_and(mask1, mask2).sum()
    iou = overlap_area / (mask1 | mask2).sum()
    return overlap_area, iou
t1, t2 = 0, 1

In [ ]:
def test_compute_overlap():
    for sc in single_cells:
        for sc_tmp in single_cells:
            # compare two overlap algorithms
            overlap_area, iou = compute_overlap(sc, sc_tmp)
            overlap_area_bf, iou_bf = compute_overlap_bf(sc, sc_tmp)
            # print(overlap_area, overlap_area_bf)
            assert overlap_area == overlap_area_bf
            assert iou == iou_bf
# test_compute_overlap()

In [ ]:
import tqdm
def compute_overlaps(sc_list1, sc_list2):
    overlap_map = {}
    for sc1 in tqdm.tqdm(sc_list1, desc="Computing overlaps"):
        for sc2 in sc_list2:
            overlap_area, iou = compute_overlap(sc1, sc2)
            overlap_map[(sc1, sc2)] = (overlap_area, iou)
    # parallel version
    return overlap_map

overlap_map_by_time = {}
times = set(sorted(list(single_cells_by_time.keys())))
max_time = max(times)
interval = 20
times_by_interval = [i for i in range(0, max_time, interval)]
times_by_interval = set(times_by_interval)
print("selected times: ", times_by_interval)
for time in times_by_interval:
    if time + 1 not in times:
        print(f"Time {time} is the last time point, skipping")
        continue
    overlap_map_by_time[time] = compute_overlaps(single_cells_by_time[time], single_cells_by_time[time + 1])

In [ ]:
sc1, sc2 = single_cells_by_time[0][0], single_cells_by_time[1][0]
overlap_map_by_time[0][(sc1, sc2)]

In [ ]:
len(single_cells_by_time[2])

Check the cells visually

In [ ]:
# for time in overlap_map_by_time:
#     overlap_map = overlap_map_by_time[time]
#     for sc_tmp1, sc_tmp2 in overlap_map:
#         if sc_tmp1 == sc_tmp2:
#             continue
#         if overlap_map[(sc_tmp1, sc_tmp2)][0] > 0:
#             print(sc_tmp1.timeframe, sc_tmp2.timeframe, overlap_map[(sc_tmp1, sc_tmp2)])
#             fig, axes = plt.subplots(1, 6, figsize=(15, 5))
#             padding=50
#             sc_tmp1.show_contour_mask(crop=False, ax = axes[0])
#             sc_tmp2.show_contour_mask(crop=False, ax = axes[1])
#             sc_tmp1.show(crop=True, ax = axes[2], padding=padding)
#             sc_tmp2.show(crop=True, ax = axes[3], padding=padding)
#             sc_tmp1.show_contour_mask(crop=True, ax = axes[4], padding=padding)
#             sc_tmp2.show_contour_mask(crop=True, ax = axes[5], padding=padding)
#             plt.show()


In [ ]:
np.unique(sc1.mask_dataset.get_img_by_time(0))

In [ ]:
np.unique(sc1.mask_dataset.get_img_by_time(1))

```
sc.datasets["img"]
sc.datasets["mask"]
sc.datasets["label"]
sc.datasets["TRITC"]
```

In [ ]:
from livecellx.segment.utils import match_mask_labels_by_iou
match_mask_labels_by_iou(sc1.mask_dataset.get_img_by_time(2), sc1.mask_dataset.get_img_by_time(1), return_all=True)

## Apply correction CNN to fix single time-frame case

In [ ]:
from livecellx.model_zoo.segmentation.sc_correction import CorrectSegNet
# ckpt = r"/home/ken67/LiveCellTracker-dev/notebooks/lightning_logs/version_real-02/checkpoints/epoch=3720-test_loss=0.0085.ckpt"
# ckpt = r"/home/ken67/LiveCellTracker-dev/notebooks/lightning_logs/version_802/checkpoints/epoch=2570-test_out_matched_num_gt_iou_0.5_percent_real_underseg_cases=0.8548.ckpt"
# ckpt = r"/home/ken67/LiveCellTracker-dev/notebooks/lightning_logs/version_v10_02/checkpoints/epoch=2999-global_step=0.ckpt"
ckpt = r"/home/ken67/LiveCellTracker-dev/notebooks/lightning_logs/version_v10_02/checkpoints/epoch=5999-global_step=0.ckpt"

model = CorrectSegNet.load_from_checkpoint(ckpt)
model = model.cuda()
model = model.eval()

In [ ]:
from livecellx.segment.ou_utils import create_ou_input_from_sc
from torchvision import transforms
from livecellx.preprocess.utils import normalize_img_to_uint8, enhance_contrast

import torch
from torchvision import transforms
from scipy import ndimage
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
import skimage
from skimage.morphology import local_maxima, h_maxima
from skimage.measure import regionprops, label


input_transforms = transforms.Compose(
        [
            transforms.Resize(size=(412, 412)),
        ]
)
from livecellx.preprocess.utils import dilate_or_erode_mask

def create_ou_input_from_sc(sc: SingleCellStatic, padding_pixels: int = 0, dtype=float, remove_bg=True, one_object=True, scale=0, bbox=None):
    if bbox is None:
        bbox = sc.get_bbox()
    if remove_bg:
        img_crop = sc.get_contour_img(padding=padding_pixels, bbox=bbox).astype(dtype)
    else:
        img_crop = sc.get_img_crop(padding=padding_pixels, bbox=bbox).astype(dtype)
    img_crop = normalize_img_to_uint8(img_crop).astype(dtype)
    if one_object:
        sc_mask = sc.get_contour_mask(padding=padding_pixels, bbox=bbox)
        sc_mask = dilate_or_erode_mask(sc_mask.astype(np.uint8), scale_factor=scale).astype(bool)
        img_crop[~sc_mask] *= -1
    else:
        img_crop[sc.get_mask_crop(padding=padding_pixels, bbox=bbox) == 0] *= -1
    return img_crop

def viz_ou_sc_outputs(sc: SingleCellStatic, model, transforms, padding_pixels: int = 0, dtype=float, remove_bg=True, one_object=True, scale=0, out_threshold=1, save_path=None, show=True):
    ou_input = create_ou_input_from_sc(sc, padding_pixels=padding_pixels, dtype=dtype, remove_bg=remove_bg, one_object=one_object, scale=scale)
    viz_ou_outputs(ou_input, sc.get_sc_mask(padding=padding_pixels, dtype=int), model, transforms, out_threshold=out_threshold, original_img=sc.get_img_crop(padding=padding_pixels), save_path=save_path, show=show)


def viz_ou_outputs(ou_input, original_mask, model, input_transforms, out_threshold, show=True, original_img=None, save_path=None):
    original_shape = ou_input.shape
    original_ou_input = ou_input.copy()
    ou_input = input_transforms(torch.tensor([ou_input]))
    ou_input = torch.stack([ou_input, ou_input, ou_input], dim=1)
    ou_input = ou_input.float().cuda()
    output = model(ou_input)

    # transform (resize) output to original shape
    back_transforms = transforms.Compose(
            [
                transforms.Resize(size=(original_shape[0], original_shape[1])),
            ]
    )
    output = back_transforms(output)

    # perform watershed on output
    marker_method = "hmax"
    h_threshold = 20

    # marker_method = "local"
    # peak_distance = 50
    markers = None
    edt_distance = output.cpu().detach().numpy()[0, 0]
    
    if markers is None and marker_method == "hmax":
        # local_hmax = h_maxima(raw_crop, h_threshold)
        local_hmax = h_maxima(edt_distance, h_threshold)
        markers = skimage.measure.label(local_hmax, connectivity=1)
    elif markers is None and marker_method == "local":
        # use local peak as default markers
        coords = peak_local_max(edt_distance, min_distance=peak_distance, footprint=np.ones((3, 3)))
        mask = np.zeros(edt_distance.shape, dtype=bool)
        mask[tuple(coords.T)] = True
        markers, _ = ndi.label(mask)

    # labels = watershed(edt_distance, markers, mask=contour_mask)
    if show:
        fig, axes = plt.subplots(1, 2, figsize=(8, 2))
        axes[0].imshow(markers)
        axes[0].set_title("markers")
        axes[1].imshow(edt_distance)
        axes[1].set_title("edt_distance")
        plt.show()

    # watershed_mask = watershed(-edt_distance, markers, mask=original_mask)
    watershed_mask = watershed(-edt_distance, markers, mask=edt_distance > out_threshold)
    
    # visualize the input and all 3 output channels
    if show or (save_path is not None):
        if original_img is not None:
            num_figures = 8
        else:
            num_figures = 7
        fig, axes = plt.subplots(1, num_figures, figsize=(15, 5))
        axes[0].imshow(original_ou_input)
        axes[0].set_title("input")
        axes[1].imshow(output[0, 0].cpu().detach().numpy())
        axes[1].set_title("output c0")
        axes[2].imshow(output[0, 1].cpu().detach().numpy())
        axes[2].set_title("output c1")
        axes[3].imshow(output[0, 2].cpu().detach().numpy())
        axes[3].set_title("output c2")
        axes[4].imshow(original_mask)
        axes[4].set_title("original mask")
        axes[5].imshow(output[0, 0].cpu().detach().numpy() > out_threshold)
        axes[5].set_title("output c0 > 1")
        axes[6].imshow(watershed_mask)
        axes[6].set_title("watershed mask")
        if original_img is not None:
            axes[7].imshow(enhance_contrast(normalize_img_to_uint8(original_img)))
            axes[7].set_title("original img")
    if show:
        plt.show()
    if save_path is not None:
        plt.savefig(save_path)

    return output, watershed_mask

selected_sc_list = [single_cells_by_time[2][12], single_cells_by_time[2][13]]
for i, sc in enumerate(selected_sc_list):
    viz_ou_sc_outputs(sc, model, input_transforms, padding_pixels=50, dtype=float, remove_bg=True, one_object=True, scale=0, show=True)

In [ ]:
for i, sc in enumerate(selected_sc_list):
    viz_ou_sc_outputs(sc, model, input_transforms, padding_pixels=50, dtype=float, remove_bg=False, one_object=True, scale=0.1)

In [ ]:
for i, sc in enumerate(selected_sc_list):
    viz_ou_sc_outputs(sc, model, input_transforms, padding_pixels=50, dtype=float, remove_bg=False, one_object=False, scale=0.1)

[TODO] judge if a case is oversegmentation
inputs:
    an input mask with other cells included
    a corected label mask
    


## Apply correction CNN to fix cells from two time frames

In [ ]:
from livecellx.segment.utils import compute_match_label_map
from livecellx.core.parallel import parallelize

times = sorted(mask_dataset.times)
inputs = []
for idx in times_by_interval:
    t1 = idx
    if t1+1 in times:
        t2 = t1+1
    else: 
        continue
    inputs.append((t1, t2, mask_dataset))
label_match_outputs = parallelize(compute_match_label_map, inputs, None)

In [ ]:
len(label_match_outputs)

In [ ]:
multiple_maps = []
for t1, t2, label_map in label_match_outputs:
    for label in label_map:
        if len(label_map[label]) > 1:
            # print(t1, t2, label, label_map[label])
            multiple_maps.append((t1, t2, label, label_map[label]))

In [ ]:
multiple_maps

In [ ]:
t1, t2, label, label_map = multiple_maps[4]
figsize = (10, 5)

sc1 = None
for sc in single_cells_by_time[t1]:
    if sc.meta["label_in_mask"] == label:
        sc1 = sc
        sc.show_panel(figsize=figsize)

sc2, sc3 = None, None
t2_labels = list(label_map.keys())
for sc in single_cells_by_time[t2]:
    if sc.meta["label_in_mask"] == t2_labels[0]:
        sc2 = sc
        sc.show_panel(figsize=figsize)
    if sc.meta["label_in_mask"] == t2_labels[1]:
        sc3 = sc
        sc.show_panel(figsize=figsize)

In [ ]:
viz_ou_sc_outputs(sc1, model, input_transforms, padding_pixels=50, dtype=float, remove_bg=False, one_object=True, scale=0.2)

In [ ]:
time2label2sc = {}
for sc in single_cells:
    if sc.timeframe not in time2label2sc:
        time2label2sc[sc.timeframe] = {}
    label = sc.meta["label_in_mask"]
    time2label2sc[sc.timeframe][label] = sc

In [ ]:
def combine_and_create_ou_input_from_scs(scs, bbox, create_ou_kwargs={
    "padding_pixels": 0,
    "dtype": float,
    "remove_bg": False,
    "one_object": True,
    "scale": 0
}):
    assert len(scs) > 1
    create_ou_kwargs.update({"bbox": bbox})
    res_inputs =  create_ou_input_from_sc(scs[0], **create_ou_kwargs)
    for sc in scs[1:]:
        sc_input = create_ou_input_from_sc(sc, **create_ou_kwargs)
        res_inputs = np.maximum(res_inputs, sc_input)
    return res_inputs

for i in range(10):
    print(">" * 80)
    rand_map_idx = np.random.randint(len(multiple_maps))
    t1, t2, label, label_map = multiple_maps[rand_map_idx]
    sc1 = time2label2sc[t1][label]
    sc2_label = list(label_map)[0]
    sc2 = time2label2sc[t2][sc2_label]
    sc3_label = list(label_map)[1]
    sc3 = time2label2sc[t2][sc3_label]
    sc1.show_panel()
    sc2.show_panel()
    sc3.show_panel()
    padding_pixels = 0
    one_object = True
    out_threshold=3
    remove_bg = False
    viz_ou_sc_outputs(sc1, model, input_transforms, padding_pixels=padding_pixels, remove_bg=remove_bg, out_threshold=out_threshold, one_object=one_object)

    bbox = sc1.get_bbox()
    create_ou_kwargs = {
        "padding_pixels": padding_pixels,
        "dtype": float,
        "remove_bg": remove_bg,
        "one_object": one_object,
        "scale": 0
    }
    combined_input = combine_and_create_ou_input_from_scs([sc2, sc3], bbox=bbox)
    print("shape of combined input:", combined_input.shape)
    # plt.imshow(combined_input)
    print(">" * 5, "outputs of sc2 and sc3")
    viz_ou_outputs(combined_input, sc3.get_mask_crop(bbox=bbox), model, input_transforms, out_threshold=out_threshold)

In [ ]:
def is_underseg(sc, show=True, remove_bg=False, min_area=500, out_threshold=1, padding_pixels=50, one_object=True):
    ou_input = create_ou_input_from_sc(sc, padding_pixels=padding_pixels, dtype=int, remove_bg=remove_bg, one_object=one_object, scale=0)
    output, watershed_mask = viz_ou_outputs(ou_input, sc.get_sc_mask(), model, input_transforms, out_threshold=out_threshold, show=show, original_img = sc.get_sc_img())
    out_seg = output.cpu().detach().numpy()[0, 0]
    out_seg = out_seg.astype(int)
    regions = regionprops(out_seg)
    # filter regions by area
    regions = [r for r in regions if r.area > min_area]
    return len(regions) > 1

# is_underseg(sc1, show=True, remove_bg=False)



In [ ]:
# randomly select 1000 cells
rand_single_cells = random.sample(single_cells, k=6000)
min_area=100
os.makedirs("./notebook_results/ou_sc_pred_figures", exist_ok=True)

for i, sc in enumerate(rand_single_cells):
    viz_ou_sc_outputs(sc, model, input_transforms, padding_pixels=50, dtype=float, remove_bg=False, one_object=True, scale=0, save_path="./notebook_results/ou_sc_pred_figures/{}.png".format(i), show=False)
    plt.clf()
    # res = is_underseg(sc, remove_bg=False, show=False, min_area=min_area)
    # if res:
    #     is_underseg(sc, remove_bg=False, show=True, min_area=min_area)
    # print("is underseg?", res)